# I40 : Consider Coroutines to Run Many Functions Concurrently

- More about : https://www.geeksforgeeks.org/coroutine-in-python/

- Threads give Python programmers a way to run multiple functions seemingly at the same time. But there are three big problems with threads:

- They require special tools to coodinate with each other safely. This makes code that uses threads harder to reason about than procedural, single-treaded code. This complexity makes threaded code more difficult to extend and maintain over time.

- Thread requirea a lot of memory, about 8 MB per executing thread. On many computers, that amount of memory doesn't matter for a dozen threads or so. But what if you want your program to run tens of thousands of functions "simultaneously"? These functions may correspond to user requests to a server pixels oon a screen, particles in a simulation, etc. Running a thread per unique activity just won't work.
- Theads are costly to start. If you want to constantly be creating new concurrent functions and finishign them, the overhead of using threads becomes large and slows everything down.

- Python can work around all these issues with *coroutines*. Coroutines let you have many seemingly sumultaneous functions in your Python programs. They're implemented as an extension to generators. The cost of starting a generator coroutine is a function call. Once active, they each use less that 1KB of memory until they're exhausted.

- Coroutines work by enabling the code consuing a generator to send a value back into the generator function after each yield expression. The generator function receives the value passed to the send function as the result of the corresponding yield expression.

In [28]:
def my_coroutine():
    while True:
        received = yield
#         value = yield received
        print('Received:', received)
        
it = my_coroutine()
next(it)
it.send('First')
it.send('Second')
print(it.send('yield'))

Received: First
Received: Second
Received: yield
None


- The initail call to next is required to prepare the generator for receiving the first send by advancing it to the first yield expression. Together, yield and send provide generators with a standard way to vary their next yielded value in response to external input.

- For example, say you want to implement a generator coroutine that yields the minimum value it's been sent so far. Here, the bare yield prepares the coroutine with the initial minumum value sent in from the outside. Then the generator repeatedly yields the new munimum in exchange for the next value to consider.

In [16]:
def minimize():
    current = yield
    while True:
        value = yield current
        current = min(value, current)
        
it = minimize()
next(it)
print(it.send(10))
print(it.send(4))
print(it.send(22))
print(it.send(-1))

10
4
4
-1


- The generator function will seemingly run forever, making forward progress with each new call to send. Like threads, coroutines are independent functions that can consume inputs from their environment and produce resulting outputs. The difference is that coroutines pause at each yield expression in the generator function and resume after each call to send from the outside. This is the magical mechanism of coroutines.

In [32]:
def print_name(prefix):
    print("Searching prefix:{}".format(prefix))
    while True:
        name = (yield)
        if prefix in name:
            print(name)
 
# calling coroutine, nothing will happen
corou = print_name("Dear")
 
# This will start execution of coroutine and 
# Prints first line "Searchig prefix..."
# and advance execution to the first yield expression
corou.__next__()
 
# sending inputs
corou.send("Atul")
corou.send("Dear Atul")


Searching prefix:Dear
Dear Atul
